In [6]:
# imports
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K

### Set Hyperparameters
Without going into too much theory or testing many different values, here we use standard values for the batch size (which defines the number of training samples to work through before updating the model weights) and number of epochs (full presentations of the data in the training set for learning). There are 10 classes since we're considering the digits 1-10.

In [2]:
# define hyperparams
num_classes = 10
batch_size = 128
epochs = 12

#### Load the Images
The next step is to load our data set and set constant image sizes for our training process. The images sizes are fixed to (28 x 28), as the network input parameters are always constant. 

In [4]:
# Image Resolution
img_rows, img_cols = 28, 28

# Loading the data
(x_train, y_train),(x_test, y_test) = mnist.load_data() 

#### Pre-processing
In this step we need to make sure that the training data is pre-processed and tuned to the same direction; if your inputs are of different sizes, the performance of your network will be inaccurate. 

We use a simple reshape method on every image and iterate it over the complete data set. Next, we assign the respected label to every image for the training process, in this case, we use the to_categorical method to assign a label to every image.

In [5]:
# Prepare data

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


#### Define the Model Architecture

In [9]:
# Creating the Model 

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.legacy.Adadelta(),
              metrics=['accuracy'])



In [10]:
# Train the model
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))


Epoch 1/12
469/469 [==============================] - 29s 61ms/step - loss: 2.2831 - accuracy: 0.1324 - val_loss: 2.2567 - val_accuracy: 0.1802
Epoch 2/12
469/469 [==============================] - 28s 60ms/step - loss: 2.2393 - accuracy: 0.2162 - val_loss: 2.2024 - val_accuracy: 0.3588
Epoch 3/12
469/469 [==============================] - 28s 59ms/step - loss: 2.1831 - accuracy: 0.3039 - val_loss: 2.1296 - val_accuracy: 0.4534
Epoch 4/12
469/469 [==============================] - 28s 59ms/step - loss: 2.1053 - accuracy: 0.3836 - val_loss: 2.0303 - val_accuracy: 0.5564
Epoch 5/12
469/469 [==============================] - 28s 59ms/step - loss: 2.0024 - accuracy: 0.4584 - val_loss: 1.8969 - val_accuracy: 0.6470
Epoch 6/12
469/469 [==============================] - 28s 59ms/step - loss: 1.8689 - accuracy: 0.5224 - val_loss: 1.7268 - val_accuracy: 0.7176
Epoch 7/12
469/469 [==============================] - 28s 59ms/step - loss: 1.7107 - accuracy: 0.5706 - val_loss: 1.5324 - val_accuracy:

#### Model Evaluation

In [11]:
# Evaluating the Predictions on the Model

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7997066378593445
Test accuracy: 0.843999981880188


#### Save model

In [15]:
# Saving the model for Future Inferences

model_json = model.to_json()
with open("../models/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("../models/model.h5")